In [36]:
import numpy as np
import matplotlib.pyplot as plt

In [37]:
dct_s = np.load('/vision/u/chpatel/dstest/dann_orig/data_train.npz', allow_pickle=True)
dct_t = np.load('/vision/u/chpatel/dstest/dann_orig/data_val.npz', allow_pickle=True)
dct_tt = np.load('/vision/u/chpatel/dstest/dann_orig/data_test.npz', allow_pickle=True)
dct_s = {k: dct_s[k] for k in dct_s.keys()}
dct_t = {k: dct_t[k] for k in dct_t.keys()}
dct_tt = {k: dct_tt[k] for k in dct_tt.keys()}

In [ ]:
def get_sb_sw(f, y, C):
    n = np.stack([(y == i).sum() for i in range(C)])
    mu_c = np.stack([f[y == i].mean(0) for i in range(C)])
    mu_c[np.isnan(mu_c)] = 0
    mu = f.mean(0)

    sb = (mu_c - mu[None]).T @ ((mu_c - mu[None]) * n_s[:, None])
    
    sw = np.zeros_like(sb)
    for i in range(C):
        fi = f[y == i]
        sw += (fi - mu_c[i][None]).T @ (fi - mu_c[i][None])
    
    return sb, sw

In [ ]:
def get_jw(sb, sw):
    swinv = np.linalg.inv(sw)
    u, s, vh = np.linalg.svd(swinv @ sb)
    w = u
    nume = np.trace(w.T @ sb @ w)
    denom = np.trace(w.T @ sw @ w)
    return nume / denom

In [ ]:
def get_jw_s_t(dct_s, dct_t):
    idx = np.arange(dct_s['epoch_feats'].shape[0], dtype=int)
    # np.random.shuffle(idx)
    # idx = idx[:10000]
    f_s = dct_s['epoch_feats'][idx]
    y_s = dct_s['epoch_y_true'][idx]

    idx = np.arange(dct_t['epoch_feats'].shape[0], dtype=int)
    # np.random.shuffle(idx)
    # idx = idx[:10000]
    f_t = dct_t['epoch_feats'][idx]
    y_t = dct_t['epoch_y_true'][idx]
    
    C = dct_s['epoch_cls_logits'].shape[1]
    
    sb_s, sw_s = get_sb_sw(f_s, y_s, C)
    jw_s = get_jw(sb_s, sw_s)
    
    sb_t, sw_t = get_sb_sw(f_t, y_t, C)
    jw_t = get_jw(sb_t, sw_t)
    
    return jw_s, jw_t

In [41]:
dctc_s = np.load('/vision/u/chpatel/dstest/cdan/data_train.npz', allow_pickle=True)
dctc_t = np.load('/vision/u/chpatel/dstest/cdan/data_val.npz', allow_pickle=True)
dctc_s = {k: dctc_s[k] for k in dctc_s.keys()}
dctc_t = {k: dctc_t[k] for k in dctc_t.keys()}

# jwc_s, jwc_t = get_jw_s_t(dctc_s, dctc_t)

In [54]:
dctce_s = np.load('/vision/u/chpatel/dstest/cdane/data_train.npz', allow_pickle=True)
dctce_t = np.load('/vision/u/chpatel/dstest/cdane/data_val.npz', allow_pickle=True)
dctce_s = {k: dctce_s[k] for k in dctce_s.keys()}
dctce_t = {k: dctce_t[k] for k in dctce_t.keys()}

In [42]:
dctb_s = np.load('/vision/u/chpatel/dstest/dann_bsp/data_train.npz', allow_pickle=True)
dctb_t = np.load('/vision/u/chpatel/dstest/dann_bsp/data_val.npz', allow_pickle=True)
dctb_s = {k: dctb_s[k] for k in dctb_s.keys()}
dctb_t = {k: dctb_t[k] for k in dctb_t.keys()}

# jwb_s, jwb_t = get_jw_s_t(dctb_s, dctb_t)

In [ ]:
print(jw_s, jw_t)
print(jwc_s, jwc_t)
print(jwb_s, jwb_t)

In [ ]:
def get_feat_singular_vals(dct):
    idx = np.arange(dct['epoch_feats'].shape[0], dtype=int)
    np.random.shuffle(idx)
    idx = idx[:1000]
    f = dct['epoch_feats'][idx]
    u, s, vh = np.linalg.svd(f)
    s = s / s.max()
    return s

In [ ]:
sv_s = get_feat_singular_vals(dct_s)
sv_t = get_feat_singular_vals(dct_t)
svb_s = get_feat_singular_vals(dctb_s)
svb_t = get_feat_singular_vals(dctb_t)

In [ ]:
k = 10
plt.plot(sv_s[:k], label='dann source',  color='green', linestyle='dashed')
plt.plot(sv_t[:k], label='dann target',  color='green', linestyle='solid')
plt.plot(svb_s[:k], label='bsp source',  color='red', linestyle='dashed')
plt.plot(svb_t[:k], label='bsp target',  color='red', linestyle='solid')
plt.legend()

In [ ]:
def get_a_dist(dct_s, dct_t):
    dy = np.concatenate([
        dct_s['epoch_metadata'][:,-1],
        dct_t['epoch_metadata'][:,-1],
    ])
    dyhat = np.concatenate([
        dct_s['epoch_domcls_logits'].argmax(-1),
        dct_t['epoch_domcls_logits'].argmax(-1),
    ])
    acc = (dy==dyhat).mean()
    err = 1-acc
    return 2*(1-2*err)

In [ ]:
print(get_a_dist(dct_s, dct_t))
print(get_a_dist(dctb_s, dctb_t))

In [66]:
import torch
import torch.nn as nn
import tqdm.notebook as tqdm

class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(2048, 1048),
            nn.BatchNorm1d(1048),
            nn.ReLU(),
#             nn.Linear(1048, 1048),
#             nn.BatchNorm1d(1048),
#             nn.ReLU(),
            nn.Linear(1048, 1)
        )
    def forward(self, x):
        return self.model(x)

def get_a_dist(dct, dct2):
    
    bs = 64
    epochs = 5
    lr = 1.e-3
    
    tt = max(dct['epoch_feats'].shape[0], dct2['epoch_feats'].shape[0])
    
    x = torch.from_numpy(np.concatenate([
        dct['epoch_feats'][:tt],
        dct2['epoch_feats'][:tt],
    ])).float().cuda()
    y = torch.from_numpy(np.concatenate([
        dct['epoch_metadata'][:tt,-1],
        dct2['epoch_metadata'][:tt,-1],
    ])).long().cuda()
                         
    class CustomDataset(torch.utils.data.Dataset):
        def __init__(self, x, y):
            super().__init__()
            self.x = x
            self.y = y

        def __getitem__(self, idx):
            return self.x[idx], self.y[idx]

        def __len__(self):
            return len(self.x)
    
    print(y.float().mean())
    model = MLP().cuda()
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1.e-4)
    
    ds = CustomDataset(x, y)
    dsloader = torch.utils.data.DataLoader(ds, batch_size=bs, drop_last=True, shuffle=True)
    
    for e in range(epochs):
        acc_sum = 0
        acc_total = 0
        for xb, yb in tqdm.tqdm(dsloader):
            
            yhb = model(xb).squeeze()
            loss = torch.nn.functional.binary_cross_entropy_with_logits(yhb, yb.float())
            optimizer.zero_grad()
            loss.mean().backward()
            optimizer.step()
            
            acc_sum += ((yhb > 0).long() == yb).sum().item()
#             acc_sum += (yhb.argmax(-1) == (torch.rand_like(yb) > 0.5).long()).sum().item()
            acc_total += yhb.shape[0]
        acc = 1.0* acc_sum / acc_total
        err = 1-acc
        print(acc, err, 2*(1-2*err))


In [67]:
get_a_dist(dct_s, dct_t)

tensor(0.8967, device='cuda:0')


  0%|          | 0/2261 [00:00<?, ?it/s]

0.9498631689517912 0.05013683104820876 1.799452675807165


  0%|          | 0/2261 [00:00<?, ?it/s]

0.9692268354710305 0.0307731645289695 1.876907341884122


...

0.9750386996904025 0.024961300309597534 1.9001547987616099


  0%|          | 0/2261 [00:00<?, ?it/s]

0.9789363113666519 0.021063688633348066 1.9157452454666077


  0%|          | 0/2261 [00:00<?, ?it/s]

0.9813273993808049 0.018672600619195068 1.9253095975232197


In [68]:
get_a_dist(dctc_s, dctc_t)

tensor(0.8967, device='cuda:0')


  0%|          | 0/2261 [00:00<?, ?it/s]

0.960733635559487 0.03926636444051301 1.842934542237948


  0%|          | 0/2261 [00:00<?, ?it/s]

0.9778789805395842 0.02212101946041578 1.9115159221583369


...

0.9829168509509066 0.01708314904909336 1.9316674038036266


  0%|          | 0/2261 [00:00<?, ?it/s]

0.9856811145510835 0.014318885448916463 1.9427244582043341


  0%|          | 0/2261 [00:00<?, ?it/s]

0.9874502432551968 0.012549756744803187 1.9498009730207873


In [69]:
get_a_dist(dctb_s, dctb_t)

tensor(0.8967, device='cuda:0')


  0%|          | 0/2261 [00:00<?, ?it/s]

0.9329942503317117 0.06700574966828832 1.7319770013268467


  0%|          | 0/2261 [00:00<?, ?it/s]

0.9524201127819549 0.04757988721804507 1.8096804511278197


...

0.9610238832375055 0.03897611676249446 1.8440955329500222


  0%|          | 0/2261 [00:00<?, ?it/s]

0.9660963069438302 0.033903693056169804 1.8643852277753208


  0%|          | 0/2261 [00:00<?, ?it/s]

0.9701044891640866 0.029895510835913353 1.8804179566563466


In [70]:
get_a_dist(dctce_s, dctce_t)

tensor(0.8967, device='cuda:0')


  0%|          | 0/2261 [00:00<?, ?it/s]

0.9641060371517027 0.03589396284829727 1.856424148606811


  0%|          | 0/2261 [00:00<?, ?it/s]

0.9794062361786819 0.020593763821318056 1.9176249447147278


...

0.9836148275099513 0.01638517249004867 1.9344593100398053


  0%|          | 0/2261 [00:00<?, ?it/s]

0.9863860017691287 0.013613998230871283 1.9455440070765149


  0%|          | 0/2261 [00:00<?, ?it/s]

0.9885490380362671 0.011450961963732853 1.9541961521450686
